# Export properties from Wikidata

Used [Protege online](https://webprotege.stanford.edu/) to model the required classes and properties.

The resulting model was saved in OWL/XMLformat (`data/knowledge-graph.owl`).

Used [WebVOWL](http://vowl.visualdataweb.org/webvowl.html) to visualize the model.

![title](data/knowledge-graph.owl.svg)

1. Start a local wikibase, e.g. using [MaRDI4NFDI/portal-compose](https://github.com/MaRDI4NFDI/portal-compose)
2. Increase memory limit by setting `ini_set( 'memory_limit', '2G' );` in LocalSettings.d/LocalSettings.override.php

Possible ways to import items and properties from Wikidata (see https://www.mediawiki.org/wiki/Wikibase/Importing):
* ~Enable federated properties from Wikidata by setting `$wgWBRepoSettings['federatedPropertiesEnabled'] = true;` in Localsettings.d/LocalSettings.override.php~  (potentially creates entity namimg conflicts)
* ~Use native MediaWiki export/import to load the required pages fom Wikidata by importing the XML files into the local wikibase. These files contains item and property pages that have been exported from Wikidata.~ (potentially creates entity naming conflicts)
* Use WikibaseImport to import pages from Wikidata. ([older version](https://github.com/Wikidata/WikibaseImport/tree/c1233da6b7122c55c95d9d925b8f4162de8807e7) works)
* Use a custom script to import the XML dumps from Wikidata into the local wiki without overwritting existing items and properties

Export these using the XML export functionality of Wikidata

## Property pages that need to be exported:
Export to `data/Wikidata-properties.xml`
* instance of (Property:P31)
* zbMATH author ID (Property:P1556)
* zbMATH work ID (Property:P894) 
* swMATH work ID (Property:P6830) 
* formatter URL (Property:P1630)
* DOI (Property:P356)
* describes a project that uses (Property:P4510)
* author (Property:P50)
* publication date (Property:P577)
* published in (Property:P1433) 
* publisher (Property:P123)
* title (Property:P1476)
* main subject (Property:P921) 
* language of work or name (Property:P407)
* volume (Property:P478)
* page(s) (Property:P304)
* issue (Property:P433)
* exact match (Property:P2888)

## Item pages that need to be exported:
Export to `data/Wikidata-items.xml`
* human (Q5)
* software (Q7397)
* scholarly article (Q13442814) 
* scientific publication (Q591041)
* publisher (Q2085381)
* index term (Q1128340) 
* language (Q34770)

# Example queries
Some example SPARQL queries that *should* be possible with the model and can be used to test it:
* List all papers that use a certain software
* List all papers by a given author, sort by date
* List all papers published by a certain journal, sort by date
* List all papers on a certain keyword
* List all software filtered by a given paper keyword
